In [1]:
import os
import sys

import ase.collections
import numpy as np
import copy
import yaml

import argparse

import ase.build
import ase.optimize
import ase.io.trajectory
import ase.visualize
import ase.vibrations
import logging

import fairchem.core.models.model_registry
import fairchem.core.common.relaxation.ase_utils

import matplotlib.pyplot as plt

sys.path.append(os.environ['SURFACE_THERMO_DIR'])
#sys.path.append('/home/moon/surface/surface_thermo')
import util

In [95]:
slab_file = '/home/moon/surface_thermo/results/slab/Cr_bcc110_slab.traj'
slab_file = '/home/moon/surface_thermo/results/slab/Fe_fcc111_slab.traj'
slab_file = '/home/moon/surface_thermo/results/slab/Cr2O3_z_slab.traj'

traj = ase.io.trajectory.Trajectory(slab_file)
slab = traj[-1]

In [96]:
local_cache = os.environ['FAIRCHEM_LOCAL_CACHE']
checkpoint_path = fairchem.core.models.model_registry.model_name_to_local_file(
    'GemNet-OC-S2EFS-nsn-OC20+OC22',
    # 'EquiformerV2-31M-S2EF-OC20-All+MD',
    #local_cache='/home/moon/surface/tmp/fairchem_checkpoints/'
    local_cache=local_cache
)
calc = fairchem.core.common.relaxation.ase_utils.OCPCalculator(
    checkpoint_path=checkpoint_path,
    cpu=True,
    seed=400
)


INFO:root:Checking local cache: /home/moon/surface_thermo/tmp/fairchem_checkpoints for model GemNet-OC-S2EFS-nsn-OC20+OC22
/home/moon/fairchem_env/lib/python3.12/site-packages/fairchem/core/common/relaxation/ase_utils.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the 

# mess with this until the position looks about right

In [102]:
test_system = copy.deepcopy(slab)

adsorbate_label = 'N'
adsorbate = ase.build.molecule(adsorbate_label)

# adsorbate_label = 'N2'
# adsorbate = ase.build.molecule(adsorbate_label)
# adsorbate.rotate(-90.0, v='y', center='COM')
# site = (0.85, 0.0)
# height = 1.8

# adsorbate.rotate(-90.0, v='y', center='COM')
# adsorbate.rotate(90.0, v='z', center='COM')
# site = (1.1, -0.5)
# height = 1.5

# trying to absorb N into bulk
# site = (1.1, -0.5)
site = (1.74707631e+00, 2.75179683e+00)
height = 0.0

ase.build.add_adsorbate(test_system, adsorbate, height=height, position=site)

ase.visualize.view(test_system, viewer='x3d')

# FInd the best height

In [65]:
system = copy.deepcopy(slab)
heights = np.linspace(0.0, 3.0, 15)
height_energies = np.zeros(len(heights))
test_system = copy.deepcopy(slab)
for i, height in enumerate(heights):
    ase.build.add_adsorbate(test_system, adsorbate, height=height, position=site)
    test_system.calc = calc
    height_energies[i] = test_system.get_potential_energy()
    # remove the adsorbate for the next iteration
    test_system = test_system[:len(slab)]
best_height = heights[np.argmin(height_energies)]
print(f"Best height is {best_height:.2f} Å")

ase.build.add_adsorbate(system, adsorbate, height=height, position=site)


Best height is 3.00 Å


In [103]:
system = copy.deepcopy(slab)
ase.build.add_adsorbate(system, adsorbate, height=0.0, position=site)

In [104]:
# system_trajectory_file = '/home/moon/surface_thermo/results/system/Cr_bcc110_N2/Cr_bcc110_N2_custom_0.traj'
# system_trajectory_file = '/home/moon/surface_thermo/results/system/Fe_fcc111_N2/Fe_fcc111_N2_custom_0.traj'

system_trajectory_file = '/home/moon/surface_thermo/results/system/Cr2O3_z_N/Cr2O3_z_N_custom_0.traj'
system.calc = calc
fmax = 0.05
MAXSTEP = 100
opt = ase.optimize.BFGS(system, trajectory=system_trajectory_file, append_trajectory=True)
opt.run(fmax=fmax, steps=MAXSTEP)

      Step     Time          Energy          fmax
BFGS:    0 23:32:33    -1864.369385        9.548527
BFGS:    1 23:32:44    -1865.812988        3.059178
BFGS:    2 23:32:55    -1866.156128        1.751973
BFGS:    3 23:33:07    -1866.282227        1.606528
BFGS:    4 23:33:16    -1866.334717        1.706170
BFGS:    5 23:33:26    -1866.679199        1.891906
BFGS:    6 23:33:36    -1866.982422        1.934495
BFGS:    7 23:33:46    -1867.405762        2.253308
BFGS:    8 23:33:56    -1867.892822        2.214406
BFGS:    9 23:34:06    -1868.282837        2.102408
BFGS:   10 23:34:16    -1868.634033        1.673296
BFGS:   11 23:34:26    -1868.802002        1.382930
BFGS:   12 23:34:36    -1868.911743        1.120977
BFGS:   13 23:34:47    -1869.004272        0.929955
BFGS:   14 23:34:56    -1869.156982        0.994823
BFGS:   15 23:35:06    -1869.265015        1.114278
BFGS:   16 23:35:16    -1869.530396        1.109621
BFGS:   17 23:35:26    -1869.801880        1.076852
BFGS:   18 23:

True

In [101]:
ase.visualize.view(system, viewer='x3d')

In [91]:
adsorbate_indices = [i for i in range(len(system)) if i >= len(slab)]

In [92]:
vib_dir = 'custom'
vib = ase.vibrations.Vibrations(system, name=vib_dir, indices=adsorbate_indices)
vib.clean()  # Clean previous results
vib.run()
vib.summary()
freq = vib.get_frequencies()

---------------------
  #    meV     cm^-1
---------------------
  0   18.9     152.6
  1   21.9     176.5
  2   22.6     182.3
  3   27.8     223.8
  4   28.5     230.0
  5  130.9    1056.0
---------------------
Zero-point energy: 0.125 eV


In [93]:
result = {
    'frequencies': freq.tolist(),
    'zpe': float(vib.get_zero_point_energy()),
}

result_file = system_trajectory_file.replace('.traj', '_vib.yaml')
with open(result_file, 'w') as f:
    yaml.dump(result, f, default_flow_style=False)

vib.clean()  # Clean results after the fact
os.rmdir(vib_dir)


In [94]:
# Save a picture of the relaxed system
side_pic = system_trajectory_file.replace('.traj', '_side.png')
top_pic = system_trajectory_file.replace('.traj', '_top.png')
ase.io.write(side_pic, system, rotation='-90x,0y,0z')
ase.io.write(top_pic, system, rotation='0x,0y,0z')


In [77]:
top_pic

'/home/moon/surface_thermo/results/system/Cr_bcc110_N2/Cr_bcc110_N2_custom_0_top.png'